# Lab 4: Non-portable HPE CSI Driver Capabilities

The HPE CSI Driver contains a number of practical features that aren't portable within CSI drivers and some are capabilities that are even unique for each of the backend CSPs that the HPE CSI Driver supports.

## The StorageClass

Let's inspect our `StorageClass`.

In [ ]:
kubectl get storageclass/hpe-standard -o yaml

The `.parameters.allowOverrides` is a comma-separated list of parameters that users may override by annotating the `PersistentVolumeClaim` at creation.

---
```yaml
---
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: my-override-pvc
  annotations:
    csi.hpe.com/limitMbps: "5"
    csi.hpe.com/description: "My Batch Job"
spec:
  accessModes:
  - ReadWriteOnce
  resources:
    requests:
      storage: 32Mi
  storageClassName: hpe-standard
  volumeMode: Block
```
---

The idea here is to create a throttled volume to run a batch `Job`. We don't want to impact any workloads, hence throttling the backed volume to 5MB/s.

The `Job` looks like this.

---
```yaml
---
apiVersion: batch/v1
kind: Job
metadata:
  name: my-override-job
spec:
  template:
    spec:
      containers:
      - name: dd
        image: hpestorage/ioping
        command: [ "dd" ]
        args: [ "if=/dev/xvda", "of=/dev/null", "iflag=direct", "bs=32k" ]
        volumeDevices:
          - name: my-data
            devicePath: /dev/xvda
      volumes:
      - name: my-data
        persistentVolumeClaim:
          claimName: my-override-pvc
      restartPolicy: Never
  backoffLimit: 4
```
---

Let's create the `PersistentVolumeClaim` and `Job`.

In [ ]:
kubectl create -f obj/my-override-pvc.yaml -f obj/my-override-job.yaml
kubectl wait --for=condition=complete --timeout=600s job/my-override-job

It takes a minute or so for the `Job` to complete. Check the logs once it finishes.

In [ ]:
kubectl logs job.batch/my-override-job

Throughput should be around the domain of 5MB/s. It could be less, if the environment is busy.

Now, inspecting the `StorageClass` above further, we can observe `.parameters.allowMutations`. This allows users to modify an attribute of a `PersistentVolumeClaim` once it has been provisioned. Let's change our 5MB/s to 10MB/s and observe the results on the next submitted `Job`.

In [ ]:
kubectl annotate --overwrite pvc/my-override-pvc csi.hpe.com/limitMbps=10

Next, start a new `Job`.

In [ ]:
kubectl replace --force -f obj/my-override-job.yaml
kubectl wait --for=condition=complete --timeout=600s job/my-override-job

Now, let's inspect the `Job` output.

In [ ]:
kubectl logs job.batch/my-override-job

Was the `Job` completed faster than previously?

Let's delete our `Job` and `PersistentVolumeClaim`.

In [ ]:
kubectl delete job/my-override-job pvc/my-override-pvc

# Access Modes

By default, the CSP backends supported by the HPE CSI Drivers are block storage based. That means only one `Pod` at any given time can access the volume and `PersistentVolumeClaims` need to be created with `.spec.accessMode.ReadWriteOnce`. However, the HPE CSI Driver includes a NFS Server Provisioner that allows both `ReadWriteMany` and `ReadOnlyMany` in the `PersistentVolumeClaim`. The NFS Server Provisioner will run a NFS server on top of a `ReadWriteOnce` claim on the Kubernetes cluster itself to allow access from multiple `Pods` across multiple nodes using the standard NFSv4 protocol.

The `StorageClass` parameter `nfsResources` controls whether to create a NFS `Deployment` and associated resources to serve the claim.

**Note:** All claims, including `ReadWriteOnce`, will provision resource, hence we want to control it granularly per claim.

Assume the following claim, using the "hpe-standard" `StorageClass`.

---
```yaml
---
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: my-nfs-pvc
  annotations:
    csi.hpe.com/nfsResources: "true"
spec:
  accessModes:
  - ReadWriteMany
  resources:
    requests:
      storage: 512Mi
  storageClassName: hpe-standard
```
---

It takes a while to create claim, as additional resources needs to be created (all happening transparently for the user).

Let's create the claim.

In [ ]:
kubectl create -f obj/my-nfs-pvc.yaml

Let's bring up a five replica `Deployment`.

---
```yaml
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: my-nfs-deploy
  labels:
    app: my-nfs-deploy
spec:
  replicas: 5
  selector:
    matchLabels:
      app: my-ioping
  template:
    metadata:
      labels:
        app: my-ioping
    spec:
      containers:
      - image: hpestorage/ioping
        name: ioping
        command: [ "ioping" ]
        args: [ "/data" ]
        volumeMounts:
          - name: my-data
            mountPath: /data
      volumes:
      - name: my-data
        persistentVolumeClaim:
          claimName: my-nfs-pvc
```
---

In [ ]:
kubectl create -f obj/my-nfs-deploy.yaml
kubectl rollout status deploy/my-nfs-deploy

Wait for all five `Pods` to come up.

In [ ]:
kubectl get pods -l app=my-multiping

Now, inspect the logs of each `Pod` to see what's going on.

In [ ]:
kubectl get pods -o name | xargs -n 1 kubectl logs --since=1s

At this point, it's evident that the `Pods` are accessing a NFS mountpoint hosted by a NFS server in the Kubernetes cluster. This might be obvious, but, then again, it might not be depending on the load of the cluster. Why is one `Pod` faster than the other four?

Let's tidy up.

In [ ]:
kubectl delete deploy/my-nfs-deploy pvc/my-nfs-pvc

## Summary

First, congratulations! You've concluded the [{{ BRANDINGWOD }} CSI Workshop-on-Demand](https://developer.hpe.com/hackshack/workshop/2). Second, the HPE CSI Driver has huge potential to improve business outcomes for data centric workloads if used correctly. Learn more about the different backends that are supported and how to get started by deploying your own Kubernetes cluster and installing the driver. Everything you need is available on the [SCOD](https://scod.hpedev.io/csi_driver/using.html#using_the_nfs_server_provisioner) portal.

Now, let's [conclude this workshop](5-WKSHP-CSI-Conclusion.ipynb)!

> **Learn more** 
> - NFS Server Provisioner on [HPE Storage Container Orchestrator Documentation](https://scod.hpedev.io/csi_driver/using.html#using_the_nfs_server_provisioner) (SCOD).
> - {{ BRANDING }} Community Blog: [Introducing an NFS Server Provisioner for the HPE CSI Driver for Kubernetes](https://developer.hpe.com/blog/xABwJY56qEfNGMEo1lDj/introducing-an-nfs-server-provisioner-for-the-hpe-csi-driver-for-kuberne)